In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports

In [66]:
import torch
from torch.utils.data import DataLoader, ConcatDataset
import torchvision as tn
import sklearn as sn
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from pprint import pprint

from data import GenImageDataset
from model import ResCNN

Environment

In [61]:
# Paths
ROOT_PATH = pl.Path('..')
DATA_PATH = ROOT_PATH / 'data'
CKPT_PATH = ROOT_PATH / 'ckpt'

# Hardware
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reproducibility
SEED = 7982
rng.seed(SEED)
ny.random.seed(SEED)
torch.manual_seed(SEED)
gen_numpy = ny.random.default_rng(SEED)
gen_torch = torch.Generator('cpu').manual_seed(SEED)

Load dataset

In [85]:
test_data = GenImageDataset(DATA_PATH, 'test')
valid_data = GenImageDataset(DATA_PATH, 'val')
train_data = GenImageDataset(DATA_PATH, 'train')
talid_data = ConcatDataset([train_data, valid_data])

In [68]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size, shuffle=True, prefetch_factor=4, num_workers=8, generator=gen_torch)
valid_loader = DataLoader(valid_data, batch_size, shuffle=True, prefetch_factor=4, num_workers=8, generator=gen_torch)
test_loader = DataLoader(test_data, batch_size, shuffle=False, prefetch_factor=4, num_workers=8, generator=gen_torch)

Create Model

In [84]:
import torch.nn as nn

epochs = 2
loss_fn = nn.CrossEntropyLoss()
model = ResCNN().to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=1e-4)

In [74]:
for e in range(epochs):
    for X, y in train_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optim.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, y)
        loss.backward()
        optim.step()

In [75]:
test_loader = DataLoader(test_data, batch_size, shuffle=False, prefetch_factor=4, num_workers=8, generator=gen_torch)

model.requires_grad_(False)
model = model.eval()

preds = []

for X in test_loader:
    X = X.to(DEVICE)
    pred = torch.argmax(model(X), dim=-1)
    preds.extend(pred.cpu().tolist())

In [76]:
preds

[67,
 16,
 40,
 13,
 69,
 45,
 29,
 54,
 26,
 98,
 4,
 74,
 68,
 58,
 35,
 94,
 43,
 60,
 0,
 67,
 33,
 74,
 18,
 38,
 96,
 96,
 91,
 84,
 35,
 84,
 91,
 17,
 16,
 45,
 97,
 90,
 68,
 23,
 94,
 56,
 29,
 23,
 59,
 32,
 3,
 90,
 22,
 62,
 0,
 97,
 28,
 13,
 46,
 32,
 2,
 62,
 13,
 27,
 13,
 66,
 80,
 45,
 85,
 76,
 80,
 51,
 53,
 23,
 79,
 8,
 32,
 68,
 25,
 18,
 74,
 76,
 77,
 4,
 44,
 59,
 85,
 88,
 63,
 1,
 13,
 37,
 0,
 61,
 98,
 2,
 26,
 86,
 24,
 50,
 56,
 68,
 52,
 78,
 75,
 67,
 68,
 80,
 96,
 14,
 64,
 79,
 26,
 19,
 74,
 91,
 67,
 87,
 85,
 81,
 58,
 65,
 13,
 33,
 3,
 10,
 88,
 64,
 44,
 49,
 11,
 76,
 61,
 68,
 35,
 17,
 76,
 91,
 70,
 29,
 71,
 10,
 74,
 49,
 2,
 96,
 64,
 31,
 33,
 94,
 31,
 77,
 44,
 86,
 6,
 67,
 72,
 41,
 75,
 86,
 59,
 98,
 30,
 78,
 37,
 80,
 77,
 24,
 92,
 38,
 41,
 62,
 35,
 31,
 30,
 85,
 76,
 76,
 71,
 80,
 24,
 76,
 78,
 96,
 54,
 44,
 50,
 74,
 44,
 99,
 96,
 77,
 41,
 88,
 40,
 67,
 86,
 33,
 29,
 79,
 91,
 9,
 41,
 91,
 21,
 9,
 75,
 3,
 75,
 

In [77]:
dataf = {'Image': [], 'Class': []}
dataf['Class'] = preds

In [80]:
test_data.data_['Class'] = preds

In [83]:
test_data.data_.to_csv('./submission0.csv',index=False)

In [81]:
test_data.data_

,Image,Class
0,046f61c4-b825-459a-8b2d-07503f5b94a5.png,67
1,67db001f-e287-4950-ac49-6683b493d1a4.png,16
2,9f1d36a1-f046-4c5d-9e8a-0a3758ff605c.png,40
3,5ffef91a-aaf9-4d0d-a219-83a9f5282361.png,13
4,c00af570-0000-4f8f-a3f2-c37a981bfdb1.png,69
...,...,...
4995,73c7c6c3-a76d-4ad4-aed0-8754d150cd51.png,91
4996,bdf2e242-7efc-4880-ba4e-35a28170d3ea.png,33
4997,17ebaea5-ff27-4d52-bebe-581a3083b667.png,51
4998,dfbca781-919c-432f-9f39-5591a039988e.png,15
